# Get Paper Info
These notebook will get the paper info for each speakers in the conference. The attributes are: Number of papers (by year), first paper year, number of citation (by year), country

In [1]:
%run setup.py

## Load the data

In [127]:
# Append all files in Data folder as a datafrom, and add a column for the file name without the extension and folder name
df_new  = pd.read_csv(os.path.join(folder_path,'factInvited_init.csv'), encoding='utf-8')
df_base  = pd.read_csv(os.path.join(folder_path,'factInvited.csv'), encoding='utf-8')

# Find new names
df_new = df_new[~df_new['Full name'].isin(df_base['Full name'])].reset_index(drop=True)

dblp_name = df_new['Full name'].str.replace(' ', '%20')
dblp_name = dblp_name.drop_duplicates().reset_index(drop=True)

# drop nan values from the list
dblp_name = dblp_name.dropna().reset_index(drop=True)

## Get data from dblp.org

In [23]:
df1 = pd.DataFrame(columns = ['Full name','Year','Year Count'])

for Author in tqdm(dblp_name[:8176]):
    author_list = []
    paper_list = []
    
    ##### Check if author is on dblp #####
    dblp_URL = "https://dblp.org/search?q=" + Author
    page = requests.get(dblp_URL)
    soup = BeautifulSoup(page.content, "html.parser")
    control_check = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")
    if len(control_check) > 0:
        first_author = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")[0].find_all("a",href=True)[0]['href']
        # print(Author," is on the webpage and the process continues...")
        
        ##### Get info from author #####
        author_page = requests.get(first_author)
        soup_of_speaker = BeautifulSoup(author_page.content, "html.parser")

        Year_list = []

        sections = soup_of_speaker.find(id="publ-section").find_all("div", class_="hide-body")
        for s in range(0,len(sections)): # Nu går vi ind på hver enkelt sektion
            rows_in_sections = sections[s].find_all('ul', class_="publ-list")[0].findAll(True, {"class":['year','entry inproceedings toc','entry article toc','entry incollection toc', 'entry book toc','entry editor toc','entry reference toc']})
            last_row_idx = 0
            
            for row in range(0,len(rows_in_sections)): # Nu går vi ind på hver enkelt række
                #### Append year multipliers
                if rows_in_sections[row].p == None:
                    Year_list.append(rows_in_sections[row].text)
                    paper_list.append(row-last_row_idx-1)
                    last_row_idx = row 
                if row == len(rows_in_sections)-1: # den sidste linje i hver sektion
                    paper_list.append(len(rows_in_sections)-last_row_idx-1)
            
            # remove all -1 values from the list
            paper_list = [x for x in paper_list if x >= 0]
                
        # create a dataframe
        df2 = pd.DataFrame(columns = ['Full name','Year','Year Count'])
        # append full name to the dataframe
        
        df2['Year'] = Year_list
        df2['Year Count'] = paper_list
        df2['Full name'] = Author
        df1 = df1.append(df2, ignore_index=True)
    time.sleep(1)
        

100%|██████████| 8176/8176 [9:18:30<00:00,  4.10s/it]     


In [29]:
df1.to_csv(os.path.join(folder_path,"paper_count_per_year.csv"), index=False)

## Combine data and save as factTable

In [14]:
# Load the initial factTable
df  = pd.read_csv(os.path.join(folder_path,'factProceedings_init.csv'), encoding='utf-8')
# Load the table with paper count per year
df1 = pd.read_csv(os.path.join(folder_path,'paper_count_per_year_proceedings.csv'), encoding='utf-8')

In [15]:
df1['Full name'] = df1['Full name'].str.replace('%20', ' ')
df1 = df1.drop_duplicates().reset_index(drop=True)
df1 = df1.sort_values(by=['Year'], ascending=True)

df1['Year Count'] = df1.groupby('Full name')['Year Count'].cumsum()

df1['max_year_count'] = df1.groupby('Full name')['Year Count'].transf  orm('max')

In [17]:
# create a function to get the paper count for a given name and year
def get_paper_count(name, year):
    df = df1[df1['Full name'] == name]
    paper_count = df[df['Year'] <= year]['Year Count']
    if len(paper_count) == 0:
        return 0
    return paper_count.iloc[-1]

# merge the dataframes and add a new column with the paper count
merged = pd.merge(df, df1.drop_duplicates(subset=['Full name']), on='Full name', how='left')
merged['Year Count'] = merged.apply(lambda x: get_paper_count(x['Full name'], x['Year_x']), axis=1)

# Clean up and control check
print("Number of rows in original table:",df.shape[0], " and umber of rows in merged table:",merged.shape[0])
print("Pct. of speakers whose information I was able to collect",round((df['Full name'].nunique()/df1['Full name'].nunique())*100,2),"%")
merged['max_year_count'] = merged['max_year_count'].fillna(0)
merged = merged.drop(columns=['Year_y'])
merged = merged.rename(columns={'Year_x': 'Year', 'Year Count': 'Paper Count', 'max_year_count': 'Max Paper Count'})
merged.to_csv(os.path.join(folder_path, "factProceedings.csv"))
print("The file is now saved")
merged.head(10)

Number of rows in original table: 289390  and umber of rows in merged table: 289390
Pct. of speakers whose information I was able to collect 118.31 %
The file is now saved


,Full name,Year,Conference (short),First name,Middle name,Last name,gender,Paper Count,Max Paper Count
0,Isao Shirakawa,1995,ASPDAC,isao,NaN,Shirakawa,M,26,93.0
1,Tingting Hwang,1995,ASPDAC,tingting,NaN,Hwang,F,16,107.0
2,Sanjay Dhar,1995,ASPDAC,sanjay,NaN,Dhar,M,4,5.0
3,Dave J. Gurney,1995,ASPDAC,dave,J.,Gurney,M,1,1.0
4,Michel Thill,1995,ASPDAC,michel,NaN,Thill,M,3,4.0
5,Huy Nam Nguyen,1995,ASPDAC,huy,Nam,Nguyen,M,6,14.0
6,Hiroaki Kunieda,1995,ASPDAC,hiroaki,NaN,Kunieda,M,1,59.0
7,Renato J. O.,1995,ASPDAC,renato,J.,O.,M,1,128.0
8,Vasily G. Moshnyaga,1995,ASPDAC,vasily,G.,Moshnyaga,M,5,92.0
9,Richard K. Wallace,1995,ASPDAC,richard,K.,Wallace,M,1,1.0
